In [ ]:
import pyarrow
import zipfile
import requests
from io import BytesIO
import pandas as pd
from zipfile import ZipFile
from numpy import dtype
from exp_studies_functions import (
    create_pivot,
    create_pivot_update,
    plot_pivot,
    plot_pivot_update,
    interactions_2var,
)
import statsmodels.api as sm
from statsmodels.formula.api import glm

"""
This script is used to read the zipped data from the SOA.org website and save in parquet format
for later use
"""

# Define the URL of the zip file
urls = [
    "https://cdn-files.soa.org/2019-group-ltd-exp-studies/2009-2013-gltd-consolidated-database.zip",
    "https://cdn-files.soa.org/2019-group-ltd-exp-studies/2014-2017-gltd-consolidated-database.zip",
]

# Datatype dictionary to reduce integer column size
dtype_dict = {
    "Actual_Deaths": dtype("int8"),
    "Actual_Recoveries": dtype("int8"),
    "Age_at_Disability_Broad_Group": dtype("O"),
    "Age_at_Disability_Group": dtype("O"),
    "Age_at_Disability": dtype("int8"),
    "Attained_Age_Group": dtype("O"),
    "Attained_Age": dtype("int8"),
    "Benefit_Duration_Grouped": dtype("O"),
    "Benefit_Max_Limit_Proxy": dtype("O"),
    "Calendar_Month": dtype("int8"),
    "Calendar_Year": dtype("int16"),
    "Case_Size_Group": dtype("O"),
    "Case_Size": dtype("int32"),
    "COLA_Indicator": dtype("O"),
    "Company_Size": dtype("O"),
    "Diagnosis_Category": dtype("O"),
    "Diagnosis_Code": dtype("O"),
    "Duration_Annual_Detailed": dtype("int16"),
    "Duration_Annual": dtype("O"),
    "Duration_Group": dtype("O"),
    "Duration_Month": dtype("int16"),
    "Duration_Qtr": dtype("O"),
    "Elimination_Period_Group": dtype("O"),
    "Elimination_Period": dtype("int8"),
    "Exclusions_Count": dtype("int16"),
    "Expected_Deaths": dtype("float32"),
    "Expected_Recoveries": dtype("float32"),
    "Exposures": dtype("float32"),
    "Gender": dtype("O"),
    "Gross_Indexed_Benefit_Amount_Group": dtype("O"),
    "Gross_Indexed_Benefit_Amount": dtype("int32"),
    "ICD_Type": dtype("int8"),
    "Indexed_Monthly_Salary_Group": dtype("O"),
    "Indexed_Monthly_Salary": dtype("int32"),
    "Industry": dtype("O"),
    "Integration_with_STD": dtype("O"),
    "Limited_Own_Occupation_Period_Group": dtype("O"),
    "Limited_Own_Occupation_Period": dtype("float32"),
    "Limits_Count": dtype("int8"),
    "M_N_Limit_Transition": dtype("O"),
    "Max_Out_Counts": dtype("int8"),
    "Mental_and_Nervous_Period_Group": dtype("O"),
    "Mental_and_Nervous_Period": dtype("int16"),
    "Original_Social_Security_Award_Status": dtype("O"),
    "OwnOccToAnyTransition": dtype("O"),
    "Record_ID": dtype("O"),
    "Record_Sequence": dtype("int64"),
    "Region": dtype("O"),
    "Replacement_Ratio_Group": dtype("O"),
    "Replacement_Ratio": dtype("float32"),
    "Residence_State": dtype("O"),
    "Settlement_Counts": dtype("int8"),
    "SIC_Code": dtype("int32"),
    "Social_Security_Award_Date": dtype("O"),
    "Study_ID": dtype("int64"),
    "Taxability_of_Benefits": dtype("O"),
    "Updated_Social_Security_Award_Status": dtype("O"),
    "Year_Group": dtype("int16"),
}

# Conslidate the zipped data into a single dataframe

dfs = []
for url in urls:
    # Send a GET request to the URL to download the zip file
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the contents of the zip file
        with ZipFile(BytesIO(response.content)) as z:
            # Extract the TXT file from the zip archive (assuming there's only one TXT file)
            txt_file_name = [name for name in z.namelist() if name.endswith(".txt")][0]

            # Read the TXT file into a pandas DataFrame
            df = pd.read_csv(z.open(txt_file_name), sep="\t", dtype=dtype_dict)

            # Append the DataFrame to the list
            dfs.append(df)
    else:
        print(f"Failed to download the zip file from URL: {url}")

# Consolidate the two datasets
gltd = pd.concat(dfs, ignore_index=True)
column_sort = sorted(gltd.columns)
gltd = gltd[column_sort]

In [ ]:
gltd = gltd[gltd["Calendar_Year"] >= 2015]

In [ ]:


# Define the formula for the GLM with interactions
formula = (
    "Actual_Recoveries ~ "
    "Duration_Group + "
    "Gender + "
    "Age_at_Disability_Group + "
    "Diagnosis_Category + "
    "Elimination_Period_Group + "
    "Limited_Own_Occupation_Period_Group + "
    "OwnOccToAnyTransition + "
    "Integration_with_STD + "
    "COLA_Indicator + "
    "Benefit_Duration_Grouped + "
    "Gross_Indexed_Benefit_Amount_Group + "
    "Replacement_Ratio_Group + "
    "Case_Size + "
    "Industry + "
    "Region + "
    "Elimination_Period_Group * Duration_Group + "
    "Age_at_Disability_Group * Duration_Group + "
    "Diagnosis_Category * Duration_Group + "
    "Integration_with_STD * Duration_Group + "
    "Gross_Indexed_Benefit_Amount_Group * Duration_Group + "
    "Case_Size * Duration_Group + "
    "Diagnosis_Category * Age_at_Disability_Group + "
    "OwnOccToAnyTransition * Limited_Own_Occupation_Period_Group + "
    "OwnOccToAnyTransition * Age_at_Disability_Group + "
    "OwnOccToAnyTransition * Diagnosis_Category"
)

# Fit the Poisson GLM with log link
poisson_model = glm(
    formula=formula,
    data=gltd,
    family=sm.families.Poisson(),
).fit()

# Print summary of the model
print(poisson_model.summary())

with open('poisson_glm_model.pkl', 'wb') as f:
    pickle.dump(poisson_model, f)